In [ ]:
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

In [ ]:
for category in ["balanced_data", "imbalanced_data", "real_data"
                ]:

    path= ".."
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")

        n_clusters =  np.unique(Y).shape[0]

        X = np.ceil(X).astype(np.int)


        for run in range(3):
            start = time.time()
            seed = run
            np.random.seed(seed)
            pca_data = PCA(2, random_state = seed).fit_transform(X)
            kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=seed)
            pred = kmeans.fit_predict(pca_data)

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, pred)
            NMI = np.around(normalized_mutual_info_score(Y, pred), 5)
            ss = silhouette_score(pca_data,pred)
            cal = calinski_harabasz_score(pca_data, pred)

            df.loc[df.shape[0]] = [
                dataset, ARI, NMI, ss, run, elapsed, pred, cal
            ]
            df.to_pickle(f"../output/pickle_results/{category}/{category}_pca_kmeans.pkl")

In [ ]:
df.groupby("dataset").mean()